In [1]:
import numpy as np

### Load data

In [2]:
data = np.load('points_case_1.npy')
P, P_Prime = data[:, :2], data[:, 2:]


### make variables for testing

In [152]:

N = 5
fP = P[:N,:]
fP_Prime = P_Prime[:N,:]

ft = np.array([1, -1])
fS = np.array(([0,0.5], [1,2]))

In [153]:

# Forward pass: Compute loss and prediction
loss = None
prediction = None

# TODO: Implement the forward pass to compute the predictions and loss,   #
# storing them in the variables above. Your implementation should be      #
# fully vectorized, and should not contain any loops (including map,      #
# filter, or comprehension expressions).                                  #


N = fP.shape[0]
fprediction = (fP @ fS) + ft
floss = np.sum(np.linalg.norm(fprediction - fP_Prime, axis=1)**2) / N

In [98]:
""" MSE vs L2**2
for each correspondence i (row), take the distance between
the predicted and actual and then the norm of that. 
ie distance = [d1, d2] = ([x_hat1, y_hat1] - [x'1, y'1])
L2 norm = (d1**2 + d2**2)^1/2

Then,sqaure that to get the sqaured L2 norm
squared l2 norm = L2norm **2

Each correspondence has a squared L2 norm, average across all of them

loss = (sum from i -> N l2norm**2) / N
"""
squaredl2 = np.sum(np.linalg.norm(fprediction - fP_Prime, axis=1)**2) / N

"""
you can also cut corners and get rid of the sqaure root and one of the sqaures
so it is like:

distance = [d1, d2] = ([x_hat1, y_hat1] - [x'1, y'1])
squared l2 norm = (d1*2 + d2**2)
loss = (sum from i -> N l2norm**2) / N

This is effectively the MSE!
"""

mse = np.sum(np.square(fprediction - fP_Prime)) / N

print(squaredl2, mse)

9.896338474347253 9.896338474347253


In [154]:
# df/dS = 2/N * sum i -> N: ((pi @ S + t) - p'i).T * pi

# convert to matrices
#       = 2/N * ((P @ S + t)) - P') * Pi 

# need to get dims to match to alow for (2,2)
# (2,N @ 2,N) -> (2,N * N,2) = (2,2) = df/dS
#
#       = 2/N * ((P @ S + t) - P').T @ Pi

f = fprediction - fP_Prime
fgrad_S = (2/N) * (fP.T @ f)

In [155]:
# df/dt = 2/N * sum i -> N : ((pi @ S + t) - p'i)
# 
# convert to matrices & sum along the columns since
# bias term [t1 t2] is broadcasted to columns on forward pass
#      = 2/N * np.sum((P @ S + t) - P', axis=1)
#

fgrad_t = (2/N) * (np.sum(fprediction - fP_Prime, axis = 0))

In [187]:
# updating
l = 1e-4
print(fS, ft)
fS = fS - (learning_rate * (fgrad_S))
ft = ft - (learning_rate * (fgrad_t))
print(fS, ft)

[[-1.73979432e-04  4.99511697e-01]
 [ 9.99889823e-01  1.99969087e+00]] [ 0.99946841 -1.00033653]
[[-3.47958864e-04  4.99023395e-01]
 [ 9.99779645e-01  1.99938174e+00]] [ 0.99893682 -1.00067306]


In [188]:
from fitting import affine_transform_loss

In [184]:
S = np.array(([1,0], [0,1]))
t = np.array([1,1])
S.shape, fS.shape

((2, 2), (2, 2))

In [185]:
loss, prediction, grad_S, grad_t = affine_transform_loss(P, P_Prime, S, t)

print(f"""loss: {loss}, 
            \nprediction:\n {prediction} 
            \ngrad_S:\n {grad_S}
            \ngrad_t:\n {grad_t}""")

loss: 6.0189619473945175, 
            
prediction:
 [[1.314      1.91857143]
 [1.32066667 1.91857143]
 [1.32733333 1.91857143]
 ...
 [1.88933333 1.02285714]
 [1.896      1.02285714]
 [1.90266667 1.02285714]] 
            
grad_S:
 [[1.28165387 1.45041876]
 [2.05623574 2.04412878]]
            
grad_t:
 [2.71854704 3.90926647]


### Next step: 
get the fit_affine_transform running with a logger

In [ ]:
from utils import Logger

def fit(args):
    data = np.load(args.data_file)
    X, Y = data[:, :2], data[:, 2:]
    logger = Logger(X, Y, print_every=args.print_loss_every)

    lr = args.learning_rate
    steps = args.steps
    S, t = fit_affine_transform(X, Y, logger, lr, steps)

    print('Final transform:')
    print('S = ')
    print(S)
    print('t = ')
    print(t)

    if args.loss_plot is not None:
        logger.save_loss_plot(args.loss_plot)